# Solar Panels Detection

Training and Detection of solar panels using YOLOv8 and the MAXAR WorldView-3 30 cm dataset (Germany region).

The original dataset was split into train, validation, and test. This step can be done with the notebook *solar_dataset_preparation.ipynb*

## Setup

In [ ]:
!pip -q install ultralytics # Ignore if not using Colab

## Import Libraries

In [ ]:
from ultralytics import YOLO
import os
import torch
import yaml
try:
    from google.colab import files
    from google.colab import drive
except:
    print("Some libraries were ignored.")

## User Inputs

In [ ]:
# Datasets folder
# Folder where the train, validation and test datasets are saved
datasets_folder = "datasets"

# YOLO model hyperparameters
# Target image size for training
image_size = 416
# Total number of training epochs
epochs_num = 100
# Batch size for training. -1 is auto.
batch_size = -1
# Minimum confidence threshold for detections
conf_thr = 0.2
# Intersection Over Union threshold
iou_thr = 0.7

## Code

### Train and Validation

Ignore the **Train and Validation** step if you already trained the model.

In [ ]:
# Create new folder (if does not exist) to store the datasets
if not os.path.exists(datasets_folder):
  os.mkdir(datasets_folder)

In [ ]:
# Mount drive
try:
  drive.mount("/content/drive")
  save_driver_folder = "/content/drive/MyDrive/datasets"
except:
  print("Unable to mount drive.")


In [ ]:
# Copies the Train, Validation and Test saved on datasets folder from your Drive to this VM
try:
  # Train
  !cp {save_driver_folder+"/train.zip"} {datasets_folder}

  # Validation
  !cp {save_driver_folder+"/validation.zip"} {datasets_folder}

  # Test
  !cp {save_driver_folder+"/test.zip"} {datasets_folder}
except:
  print("Copy of datasets from Drive ignored.")

In [ ]:
# Unzip the datasets
# Train
!unzip -q -o {datasets_folder+"/train.zip"} -d {datasets_folder}
os.remove(datasets_folder+"/train.zip")

# Validation
!unzip -q -o {datasets_folder+"/validation.zip"} -d {datasets_folder}
os.remove(datasets_folder+"/validation.zip")

# Test
!unzip -q -o {datasets_folder+"/test.zip"} -d {datasets_folder}
os.remove(datasets_folder+"/test.zip")

In [ ]:
# Check if GPU is available
if torch.cuda.is_available():
  print("GPU available.")
  torch.cuda.set_device(0)
  device = 0 # Single GPU, it has support for multiple
else:
  print("GPU not available, using CPU.")
  print("On Colab, go to: Edit > Notebook Settings > Hardware Accelerator > Select T4 GPU")
  torch.cuda.set_device(-1)
  device = "cpu"

In [ ]:
# Load best pre-trained YOLOv8 model
yv8x_model = YOLO("models/yolov8x.pt")

In [ ]:
# Generate yaml file
yaml_path = os.path.join(datasets_folder, "solar_panels_data.yaml")
yaml_data = {"names":["Solar panels"], "nc":1, "path":os.path.abspath("datasets"),
              "train":"train/images", "val":"validation/images", "test":"test/images"}
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(yaml_data, yaml_file, default_flow_style=False)

In [ ]:
train_results = yv8x_model.train(data=yaml_path, epochs=epochs_num, batch=batch_size,
                                 seed=42, imgsz=image_size, verbose=True, val=True, plots=True, device=device)

In [ ]:
# Copy best weights to Drive
try:
  !cp {"runs/detect/train/weights/best.pt"} {save_driver_folder}
except:
  print("Saving to drive ignored.")

In [ ]:
# Download the best weights
try:
  files.download("runs/detect/train/weights/best.pt")
except:
  print("Download to local machine ignored.")

### Test

In [ ]:
# Import best trained YOLO solar panels model
try:
  best_model = YOLO("runs/detect/train/weights/best.pt")
except:
  print("Loaded from drive.")
  !cp {save_driver_folder+"/solar_100epochs_-1batch_yolov8x_best.pt"} {datasets_folder}
  best_model = YOLO(datasets_folder+"/solar_100epochs_-1batch_yolov8x_best.pt")
  

In [ ]:
# Try the new model on the Test dataset
test_images_folder = os.path.join(datasets_folder, "test", "images")
test_results = best_model.predict(source=test_images_folder, imgsz=image_size, conf=conf_thr,
                                  iou=iou_thr, device=device, show_labels=False,
                                  show_conf=False, save=True)